In [1]:
from webscrapping.database.sql_queries import ValorantQueries
import pandas as pd

In [2]:
vq = ValorantQueries()
vq.set_match(43621)

Connected to database valorant successfully
[('teams',), ('roundevents',), ('players',), ('events',), ('series',), ('matches',), ('maps',), ('rounds',), ('roundeconomies',), ('roundlocations',), ('playermapinstance',), ('assists',)]


In [4]:
events = vq.get_event_table()
pd.DataFrame(events)

,Round,EventIndex,Round_time_millis,PlayerID,PlayerName,VictimID,VictimName,EventType,DamageType,WeaponID,...,Alive_Bazzi,Alive_Fisker,Alive_Munchkin,Alive_Saadhak,Alive_Sacy,Alive_ade,Alive_frz,Alive_gtnziN,Alive_neth,Alive_sutecas
0,1,1,17272,194,sutecas,1824,Fisker,kill,weapon,12,...,1,0,1,1,1,1,1,1,1,1
1,1,2,29339,2895,Saadhak,876,Munchkin,kill,weapon,11,...,1,0,0,1,1,1,1,1,1,1
2,1,3,29760,1877,frz,2388,neth,kill,weapon,12,...,1,0,0,1,1,1,1,1,0,1
3,1,4,37694,2895,Saadhak,0,NaN,plant,None,0,...,1,0,0,1,1,1,1,1,0,1
4,1,5,38461,1877,frz,1075,Bazzi,kill,weapon,12,...,0,0,0,1,1,1,1,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
196,22,6,93788,2388,neth,0,NaN,plant,None,0,...,0,0,0,1,1,1,0,1,1,0
197,22,7,94458,2294,Sacy,1843,ade,kill,weapon,6,...,0,0,0,1,1,0,0,1,1,0
198,22,8,107910,2388,neth,2895,Saadhak,kill,weapon,19,...,0,0,0,0,1,0,0,1,1,0
199,22,9,109289,2294,Sacy,2388,neth,kill,ability,0,...,0,0,0,0,1,0,0,1,0,0


In [14]:
events = vq.get_event_table()
player_columns = [item for item in events.columns if item.startswith('Alive_')]
for i in range(1, len(events)):
    row = events.loc[i].copy()
    event_index = row['EventIndex']
    victim_name = row['VictimName']
    event_type = row['EventType']
    if event_index == 2:
        for player in player_columns:
            row[player] = 1
    events.update(events.loc[i].replace(row))
events

,Round,EventIndex,Round_time_millis,PlayerID,PlayerName,VictimID,VictimName,EventType,DamageType,WeaponID,...,Alive_Bazzi,Alive_Fisker,Alive_Munchkin,Alive_Saadhak,Alive_Sacy,Alive_ade,Alive_frz,Alive_gtnziN,Alive_neth,Alive_sutecas
0,1,1,17272,194,sutecas,1824,Fisker,kill,weapon,12,...,0,0,0,0,0,0,0,0,0,1
1,1,2,29339,2895,Saadhak,876,Munchkin,kill,weapon,11,...,0,0,0,1,0,0,0,0,0,0
2,1,3,29760,1877,frz,2388,neth,kill,weapon,12,...,0,0,0,0,0,0,1,0,0,0
3,1,4,37694,2895,Saadhak,0,NaN,plant,None,0,...,0,0,0,1,0,0,0,0,0,0
4,1,5,38461,1877,frz,1075,Bazzi,kill,weapon,12,...,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
196,22,6,93788,2388,neth,0,NaN,plant,None,0,...,0,0,0,0,0,0,0,0,1,0
197,22,7,94458,2294,Sacy,1843,ade,kill,weapon,6,...,0,0,0,0,1,0,0,0,0,0
198,22,8,107910,2388,neth,2895,Saadhak,kill,weapon,19,...,0,0,0,0,0,0,0,0,1,0
199,22,9,109289,2294,Sacy,2388,neth,kill,ability,0,...,0,0,0,0,1,0,0,0,0,0


In [8]:
aux = vq.get_initial_state()
events = vq.get_event_table()
player_names = ["Alive_" + sub for sub in vq.player_names]
first_states = events.groupby("Round").first().reset_index()

for column_name in player_names:
    first_states[column_name] = 1

first_states

,Round,Round_time_millis,PlayerID,PlayerName,VictimID,VictimName,EventType,DamageType,WeaponID,Ability,Alive_Bazzi,Alive_Fisker,Alive_Munchkin,Alive_Saadhak,Alive_Sacy,Alive_ade,Alive_frz,Alive_gtnziN,Alive_neth,Alive_sutecas
0,1,17272,194,sutecas,1824,Fisker,kill,weapon,12,0,1,1,1,1,1,1,1,1,1,1
1,2,64105,194,sutecas,1824,Fisker,kill,weapon,5,0,1,1,1,1,1,1,1,1,1,1
2,3,30386,2895,Saadhak,0,Bazzi,plant,None,0,0,1,1,1,1,1,1,1,1,1,1
3,4,23540,2294,Sacy,1843,ade,kill,weapon,6,0,1,1,1,1,1,1,1,1,1,1
4,5,72891,2895,Saadhak,2388,neth,kill,weapon,6,0,1,1,1,1,1,1,1,1,1,1
5,6,33761,2294,Sacy,876,Munchkin,kill,weapon,4,0,1,1,1,1,1,1,1,1,1,1
6,7,7526,1824,Fisker,2294,Sacy,kill,weapon,15,0,1,1,1,1,1,1,1,1,1,1
7,8,36225,1824,Fisker,1877,frz,kill,weapon,15,0,1,1,1,1,1,1,1,1,1,1
8,9,8170,876,Munchkin,1877,frz,kill,weapon,4,0,1,1,1,1,1,1,1,1,1,1
9,10,41640,3105,gtnziN,1843,ade,kill,weapon,4,0,1,1,1,1,1,1,1,1,1,1


In [25]:
aux_a = aux[aux["Status"] == "alive"]
aux_b = aux.groupby(["Round", 'Team ID'])
agg_columns = ("Remaining Creds", "Loadout Value", "Weapon Price", 'Shield', 'Controller', 'Duelist', 'Initiator', 'Sentinel')
agg_dict = {col:'sum' for col in agg_columns}
agg_dict["Has Operator"] = "any"
aux_c = aux_b.agg(agg_dict).reset_index()
aux_c

,Round,Round_time_millis,PlayerID,PlayerName,VictimID,VictimName,EventType,DamageType,WeaponID,Ability,Alive_Bazzi,Alive_Fisker,Alive_Munchkin,Alive_Saadhak,Alive_Sacy,Alive_ade,Alive_frz,Alive_gtnziN,Alive_neth,Alive_sutecas
0,1,17272,194,sutecas,1824,Fisker,kill,weapon,12,0,1,1,1,1,1,1,1,1,1,1
1,2,64105,194,sutecas,1824,Fisker,kill,weapon,5,0,1,1,1,1,1,1,1,1,1,1
2,3,30386,2895,Saadhak,0,Bazzi,plant,None,0,0,1,1,1,1,1,1,1,1,1,1
3,4,23540,2294,Sacy,1843,ade,kill,weapon,6,0,1,1,1,1,1,1,1,1,1,1
4,5,72891,2895,Saadhak,2388,neth,kill,weapon,6,0,1,1,1,1,1,1,1,1,1,1
5,6,33761,2294,Sacy,876,Munchkin,kill,weapon,4,0,1,1,1,1,1,1,1,1,1,1
6,7,7526,1824,Fisker,2294,Sacy,kill,weapon,15,0,1,1,1,1,1,1,1,1,1,1
7,8,36225,1824,Fisker,1877,frz,kill,weapon,15,0,1,1,1,1,1,1,1,1,1,1
8,9,8170,876,Munchkin,1877,frz,kill,weapon,4,0,1,1,1,1,1,1,1,1,1,1
9,10,41640,3105,gtnziN,1843,ade,kill,weapon,4,0,1,1,1,1,1,1,1,1,1,1
